Updates to script for version 2

1.   Using PySpark & Spark Pandas API for better performance & speed
2.   Using Azure Cousmos DB to send cleaned data to PowerBI Dashboard
3.   Minor usage of Numba Framework to speed up some methods within this script
4.   Potential Trials of Time-Series Machine Learning Modeling of Data 
5.   Trials with using the Libra Library for potential simple Machine Learning development
6.   Determine if using Azure Synapse makes sense for this project for giving no-code manipulation of the data

Website Links & Documentation for all program enhancements

1.   PySpark Examples - https://github.com/apache/spark/tree/f74867bddf/examples/src/main/python
2.   Azure Cosmos DB - https://docs.microsoft.com/en-us/azure/cosmos-db/sql/create-sql-api-python
2.   Azure Cosmos DB Github Demo - https://github.com/Azure-Samples/azure-cosmos-db-python-getting-started/blob/main/cosmos_get_started.py
3.   Numba - https://numba.pydata.org/
4.   Time-Series Machine Learning Algos - https://www.advancinganalytics.co.uk/blog/2021/06/22/10-incredibly-useful-time-series-forecasting-algorithms
5.   Libra - https://libradocs.org/
6.   Azure Synapse & Cosmos DB Integration - https://docs.microsoft.com/en-us/azure/cosmos-db/synapse-link
7.   Spark SQL & Python - https://spark.apache.org/docs/latest/sql-programming-guide.html
8.   DeepHaven Streaming Dataframe - https://deephaven.io/?utm_term=streaming%20data&utm_campaign=Website+Traffic+Q4+2022&utm_source=adwords&utm_medium=ppc&hsa_acc=4673439537&hsa_cam=18322365595&hsa_grp=139855244374&hsa_ad=621564846172&hsa_src=g&hsa_tgt=kwd-161093182&hsa_kw=streaming%20data&hsa_mt=p&hsa_net=adwords&hsa_ver=3
9.   Spark SQL & Python - https://spark.apache.org/docs/latest/
10.  Google Sheets & Python - https://ploomber.io/blog/gsheets/

In [2]:
#all imports 
import gspread
import numpy as np
import pyspark
import pyspark.pandas as ps
import numba
from numba import jit
import os
import json
from datetime import *

In [3]:
#setup spark env & import spark variables
from pyspark.sql.types import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/29 01:02:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [34]:
#call to the dataset within Google Sheets
sa = gspread.service_account(filename="/com.docker.devenvironments.code/kuthesisdataclean-558333f9362c.json")
sheet = sa.open("POU_Unclean_Data")
work_sheet = sheet.worksheet("POU")
rows = work_sheet.get_all_values()
column_names = ['Item_no', 'Item Description', 'Quantity', 'Status', 'Serial Number', 'Reason Code', 'Shortage Notes', 'Team Found', 'Team Responsible', 'Data Found', 'Time Found Pacific', 'Data Found Pacific', 'SHORT_FOUND_EMP_ID', 'Date_chassis_start', 'date_veh_offln', 'date_mfg_rlse', 'PRIM_SHOP_ADDR', 'SCD_SHOP_ADDR_1', 'SCD_SHOP_ADDR_2', 'SCD_SHOP_ADDR_3']
#rows = rows.get_all_records()
df = spark.createDataFrame(rows)

In [ ]:
#Send data to the Cosmos DB instance
cosmosEndpoint = "https://REPLACEME.documents.azure.com:443/"
cosmosMasterKey = "REPLACEME"
cosmosDatabaseName = "sampleDB"
cosmosContainerName = "sampleContainer"

cfg = {
  "spark.cosmos.accountEndpoint" : cosmosEndpoint,
  "spark.cosmos.accountKey" : cosmosMasterKey,
  "spark.cosmos.database" : cosmosDatabaseName,
  "spark.cosmos.container" : cosmosContainerName,
}